# Your First Storm API Call

Let's make your first API call to Storm and explore the basic functionality.

## Setup

First, let's import required libraries and set up our configuration.

In [ ]:
import requests
import json
import os
from datetime import datetime

# Configuration
API_KEY = os.getenv("STORM_API_KEY", "your-api-key-here")
API_URL = "https://live-stargate.sionic.im"

# Headers
headers = {
    "storm-api-key": API_KEY
}

print(f"✅ Storm API configured")
print(f"📍 API URL: {API_URL}")
print(f"🔑 API Key: {API_KEY[:10]}..." if API_KEY != "your-api-key-here" else "⚠️  Please set your API key")

## 1. List Agents

Let's start by listing available agents - this is the simplest API call.

In [ ]:
# List agents
response = requests.get(
    f"{API_URL}/api/v2/agents",
    headers=headers,
    params={"page": 1, "size": 10}
)

if response.status_code == 200:
    data = response.json()
    agents = data["data"]["data"]
    page_info = data["data"]["pageInfo"]
    
    print(f"✅ Successfully retrieved agents!")
    print(f"\n📊 Found {page_info['totalElements']} total agents")
    print(f"📄 Showing page {page_info['page']} of {page_info['totalPages']}")
    
    print("\n🤖 Agents:")
    for agent in agents:
        print(f"  • {agent['name']} (ID: {agent['id']})")
        print(f"    Language: {agent.get('language', 'N/A')}")
        print(f"    Created: {agent['createdAt'][:10]}")
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

## 2. Create Your First Bucket

Buckets organize your documents. Let's create one.

In [ ]:
# Select first agent or use a specific ID
if 'agents' in locals() and agents:
    agent_id = agents[0]['id']
else:
    agent_id = "your-agent-id"  # Replace with actual agent ID

# Create a bucket
bucket_name = f"test-bucket-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

response = requests.post(
    f"{API_URL}/api/v2/buckets",
    headers=headers,
    json={
        "agentId": agent_id,
        "name": bucket_name
    }
)

if response.status_code == 200:
    bucket = response.json()["data"]
    print(f"✅ Bucket created successfully!")
    print(f"\n📦 Bucket Details:")
    print(f"  • ID: {bucket['id']}")
    print(f"  • Name: {bucket['name']}")
    print(f"  • Type: {bucket.get('type', 'default')}")
    print(f"  • Created: {bucket['createdAt']}")
    
    bucket_id = bucket['id']  # Save for later use
else:
    print(f"❌ Error creating bucket: {response.status_code}")
    print(response.text)

## 3. List Buckets

Let's verify our bucket was created by listing all buckets.

In [ ]:
# List buckets for the agent
response = requests.get(
    f"{API_URL}/api/v2/buckets",
    headers=headers,
    params={
        "agentId": agent_id,
        "page": 1,
        "size": 20
    }
)

if response.status_code == 200:
    data = response.json()
    buckets = data["data"]["data"]
    
    print(f"✅ Found {len(buckets)} buckets")
    print("\n📦 Buckets:")
    for bucket in buckets[:5]:  # Show first 5
        print(f"  • {bucket['name']}")
        print(f"    ID: {bucket['id']}")
        print(f"    Created: {bucket['createdAt'][:10]}")
        print()

## 4. Understanding Response Structure

Storm API responses follow a consistent structure. Let's examine it.

In [ ]:
# Make a simple request
response = requests.get(
    f"{API_URL}/api/v2/agents",
    headers=headers,
    params={"page": 1, "size": 1}
)

if response.status_code == 200:
    data = response.json()
    
    print("📋 Response Structure:")
    print("\n1. Top Level:")
    print(f"   • status: {data['status']}")
    print(f"   • data: {type(data['data'])}")
    
    print("\n2. Data Level:")
    print(f"   • data: {type(data['data']['data'])} (actual content)")
    print(f"   • pageInfo: {type(data['data']['pageInfo'])} (pagination)")
    
    print("\n3. Page Info:")
    page_info = data['data']['pageInfo']
    for key, value in page_info.items():
        print(f"   • {key}: {value}")
    
    print("\n💡 Success responses always have:")
    print("   • status: 'success'")
    print("   • data: containing the response payload")

## 5. Error Handling

Let's see how Storm API handles errors.

In [ ]:
# Try an invalid request
print("Testing error handling...\n")

# 1. Missing authentication
response = requests.get(f"{API_URL}/api/v2/agents")
print(f"1. No auth header: Status {response.status_code}")

# 2. Invalid endpoint
response = requests.get(
    f"{API_URL}/api/v2/invalid-endpoint",
    headers=headers
)
print(f"2. Invalid endpoint: Status {response.status_code}")

# 3. Invalid parameters
response = requests.get(
    f"{API_URL}/api/v2/agents",
    headers=headers,
    params={"page": -1}  # Invalid page number
)
print(f"3. Invalid params: Status {response.status_code}")

# Error response structure
if response.status_code != 200:
    print("\n❌ Error Response Structure:")
    try:
        error_data = response.json()
        print(json.dumps(error_data, indent=2))
    except:
        print(response.text)

## 6. Helper Functions

Let's create reusable helper functions for common operations.

In [ ]:
class StormAPIClient:
    """Simple Storm API client with common operations."""
    
    def __init__(self, api_key, base_url="https://live-stargate.sionic.im"):
        self.api_key = api_key
        self.base_url = base_url
        self.headers = {"storm-api-key": api_key}
    
    def _request(self, method, endpoint, **kwargs):
        """Make HTTP request with error handling."""
        url = f"{self.base_url}{endpoint}"
        
        response = requests.request(
            method, url, headers=self.headers, **kwargs
        )
        
        if response.status_code != 200:
            print(f"❌ Error {response.status_code}: {response.text}")
            return None
        
        data = response.json()
        if data.get("status") != "success":
            print(f"❌ API Error: {data}")
            return None
        
        return data["data"]
    
    def list_agents(self, page=1, size=10):
        """List agents with pagination."""
        return self._request(
            "GET", "/api/v2/agents",
            params={"page": page, "size": size}
        )
    
    def create_bucket(self, agent_id, name):
        """Create a new bucket."""
        return self._request(
            "POST", "/api/v2/buckets",
            json={"agentId": agent_id, "name": name}
        )
    
    def list_buckets(self, agent_id, page=1, size=10):
        """List buckets for an agent."""
        return self._request(
            "GET", "/api/v2/buckets",
            params={"agentId": agent_id, "page": page, "size": size}
        )

# Initialize client
client = StormAPIClient(API_KEY)

# Test the client
print("Testing Storm API Client...\n")

# List agents
agents_data = client.list_agents(page=1, size=5)
if agents_data:
    print(f"✅ Found {len(agents_data['data'])} agents")
    
    # Create a test bucket
    if agents_data['data']:
        agent_id = agents_data['data'][0]['id']
        bucket = client.create_bucket(
            agent_id, 
            f"client-test-{datetime.now().strftime('%H%M%S')}"
        )
        if bucket:
            print(f"✅ Created bucket: {bucket['name']}")